In [1]:
# API KEY Loading
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser


# DatetimeOutputParser

1. 출력 형식 정의: 
    - 텍스트 형태의 날짜/시간일 경우 이를 Python의 `datetime` 객체로 파싱   
    - (eg) `"2024-12-25"` --> `datetime.date(2024, 12, 25)`
2. 유연한 포맷 지원: 
    - 다양한 날짜/시간 포맷 처리 가능  
    - (eg) `"25th December 2024"`, `"2024/12/25 10:30 AM"`
    

[Reference] https://python.langchain.com/api_reference/langchain/output_parsers/langchain.output_parsers.datetime.DatetimeOutputParser.html

** Parser가 필요한 경우 ** 
1. 유횽성 검증:
    - AI모델의 출력이 정확한 날짜/시간 형식인지 확인 
    - AI가 예상치 못한 텍스트를 출력하면, 파서에서 예외를 발생시켜 오류를 처리할 수 있음 
    - (예) 모델 출력이 `대한민국의 광복기념일은 8월 15일 입니다.` 와 같이 텍스트인 경우, 날짜 부분만 파싱하고 나머지는 무시 
2. 추가적인 날짜 연산:
    - 모델 응답을 파싱한 후 날짜 비교/차이 계산, 날짜 추가 등의 작업을 수행할 때 유용

** Parser가 필요하지 않은 경우 **
1. 모델 출력이 이미 날짜 포맷인 경우:
    - AI모델이 항상 `"YYYY-MM-DD"` 형식으로 응답하도록 프롬프트를 구성 
2. 추가 유효성 검증 또는 연산 불필요:
    - 단순히 날짜 출력하는 작업만 수행한다면, 문자열을 파이썬 포맷팅 통해 처리하는 것으로 충분

**참고**

`2024-07-04 14:45:08`

| 형식 코드 | 설명                | 예시          |
|------------|---------------------|---------------|
| %Y         | 4자리 연도          | 2024          |
| %y         | 2자리 연도          | 24            |
| %m         | 2자리 월            | 07            |
| %d         | 2자리 일            | 04            |
| %H         | 24시간제 시간       | 14            |
| %I         | 12시간제 시간       | 02            |
| %p         | AM 또는 PM          | PM            |
| %M         | 2자리 분            | 45            |
| %S         | 2자리 초            | 08            |
| %f         | 마이크로초 (6자리)  | 000123        |
| %z         | UTC 오프셋          | +0900         |
| %Z         | 시간대 이름         | KST           |
| %a         | 요일 약어           | Thu           |
| %A         | 요일 전체           | Thursday      |
| %b         | 월 약어             | Jul           |
| %B         | 월 전체             | July          |
| %c         | 전체 날짜와 시간     | Thu Jul  4 14:45:08 2024 |
| %x         | 전체 날짜           | 07/04/24      |
| %X         | 전체 시간           | 14:45:08      |

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import DatetimeOutputParser

In [20]:
# 모델 정의
model = ChatOpenAI(temperature=0)

In [21]:
# 파서 정의
output_parser = DatetimeOutputParser()
print(output_parser.get_format_instructions())

Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 1168-07-14T17:34:59.318192Z, 1718-11-16T01:30:33.528671Z, 0928-09-06T21:58:46.173981Z

Return ONLY this string, no other words!


In [22]:
# 템플릿 정의
template = """Answer the users question:\n\n
#Format Instructions: \n{format_instructions}\n\n
# #Question: \n{question}\n\n
# #Answer:
"""

# 프롬프트 정의 
prompt = PromptTemplate.from_template(
    template,
    partial_variables = {
        'format_instructions': output_parser.get_format_instructions(),
    }
)

# 체인 정의
chain = prompt | model | output_parser

In [38]:
# 실행 
answer = chain.invoke({'question':'대한민국는 언제 광복을 맞이했나요?'})
output = answer.strftime("%Y-%m-%d")
print(answer, output)

1945-08-15 00:00:00 1945-08-15


In [40]:
# 실행 
answer = chain.invoke({'question':'올해는 대한민국 광복 이후 몇년이 지났나요?'})
print(answer)

OutputParserException: Could not parse datetime string: 1945-2022-01T00:00:00.000000Z
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

날짜 연산까지 되도록 처리하려면...

In [41]:
from datetime import datetime

In [42]:
# 날짜 파싱 및 연산 함수 추가
def calculate_years_since(event_date_str, current_year):
    try:
        # 문자열을 datetime 객체로 변환
        event_date = datetime.strptime(event_date_str, "%Y-%m-%d")
        return current_year - event_date.year
    except ValueError as e:
        raise ValueError(f"Invalid date format: {event_date_str}") from e

In [43]:
# 1. 첫 번째 질문: 광복일 확인
answer_1 = chain.invoke({'question': '대한민국는 언제 광복을 맞이했나요?'})

# 2. 광복 이후 몇 년이 지났는지 계산
try:
    # 현재 연도 계산
    current_year = datetime.now().year

    # 광복 이후 몇 년 경과했는지 계산
    years_since = calculate_years_since(answer_1.strftime("%Y-%m-%d"), current_year)
    print(f"광복 이후 경과한 년수: {years_since}년")
except Exception as e:
    print(f"Error: {e}")

광복 이후 경과한 년수: 80년


-----
** End of Documents **